In [1]:
from data.data import Data
from model.model import *

data_raw = Data().get_all_data()

df = data_raw['MBTI 500']
df = df[['type', 'posts']]
df.rename(columns={'posts': 'text'}, inplace=True)
df = df.sample(n = 10000, ignore_index = True, random_state = 1)

In [2]:
from scripts.Preprocessing_full import training_preprocessing, training_oversampling, training_balancing, training_vectorize


df2a = training_preprocessing(df)
df3 = training_oversampling(df2a)
df4 = training_balancing(df3)
df_list = training_vectorize(df4)


Dataset contains 122553 rows

cleaned dataset contains 50000 rows and 8 columns


: 

: 

In [ ]:
from model.model import *
model = initialize_model()

hist = train_model(df_list, model)

hist

In [ ]:
# ENTP #

df = data_raw['MBTI 500']
df = df[['type', 'posts']]
df.rename(columns={'posts': 'text'}, inplace=True)
df = df.sample(n = 1000, ignore_index = True, random_state = 1)

df_pred = df.iloc[:1].drop(columns =['type'])
df_pred

In [ ]:
from scripts.Preprocessing_full import prediction_preprocessing, prediction_vectorize

## Current Step for Work, error is lack of terms?

df_pred_pp = prediction_preprocessing(df_pred)
df_pred_list = prediction_vectorize(df_pred_pp)

In [ ]:
df_pred_list[0]

In [ ]:
# Making sure to run the 4 models with the 4 dataframes, again a pipeline makes more sense
 
# When predicting, the model needs to see the same features as the original fit
# So with new text, how do we pass the new features? 
# Would uploading a pickle file help?
 
predict_model([model,model,model,model], df_pred_list)